In [1]:
import numpy as np
import plotly.express as px
import pandas as pd
import math
import string
from collections import Counter
import sys

printable = set(string.printable)
printable


{'\t',
 '\n',
 '\x0b',
 '\x0c',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~'}

In [2]:
# with open('Jack_London_-_The_Sea_Wolf.txt') as inp:
#     with open('Jack_London_-_The_Sea_Wolf_ascii.txt', 'a') as outp:
#         for line in inp:
#             outp.write(filter(lambda x: x in printable, line) + '\n')


In [3]:
with open('Jack_London_-_The_Sea_Wolf_ascii.txt', 'r') as file:
    text = file.read()


In [4]:
len(text)


577875

Правила:
    1. Заглавная после точки и пробела
    

In [5]:
sum(c.isupper() for c in text)


12142

По правилу - заглавная, в действительности - маленькая

In [6]:
import re
low_char_aft_dot = []
p = re.compile("\. [a-z]")
for m in p.finditer(text):
    low_char_aft_dot.append(m.start()+2)
    # print(m.start()+2, m.group())


In [7]:
low_char_aft_dot


[210266, 371455]

По правилу - маленькая, в действительности - заглавная 

r'(\. [a-z])|( "[a-z])|(\t[a-z])'

In [8]:
class TextPreprocessor:
    def decap(self, text):
        upper_inst_lower = []
        lower_inst_upper = []
        for m in re.compile(r'((?!\. )[A-Z])|'
                            # + r'((?!\! )[A-Z])|'
                            + r'((?!\t)[A-Z])').finditer(text):
            upper_inst_lower.append(m.start())  # TODO ? !
        for m in re.compile(
            r'(\. [a-z])|' +
            # r'(\! [a-z])|' +
            r'(\t[a-z])'
        ).finditer(text):
            lower_inst_upper.append(m.end()-1)
        exceptions = lower_inst_upper + upper_inst_lower
        exceptions_bitmap = [0]*len(text)
        for e in exceptions[:40]:
            # print(text[e-2:e+2])
            pass
        for e in exceptions:
            exceptions_bitmap[e] = 1
        decap = text.lower()
        return decap, exceptions_bitmap
    # TODO gamma code(bitmap) & decode


In [9]:
from collections import Counter, defaultdict

decap, exceptions_bitmap = TextPreprocessor().decap(text)
cntr = Counter(exceptions_bitmap)
cntr[1]/cntr[0]


0.021466032442980853

### bitmap as array of numbers

In [10]:
DIGITS_IN_NUBER = 8


In [11]:
"""Add extra zeros to the end of bitmap array so it will have length divisable by DIGITS_IN_NUBER"""
extra_zeros = (DIGITS_IN_NUBER - (len(exceptions_bitmap) %
               DIGITS_IN_NUBER)) % DIGITS_IN_NUBER
bitmap = exceptions_bitmap + [0]*extra_zeros


In [12]:
def gamma_code(number_bits):
    number = int(''.join(map(str, number_bits)), 2)
    number += 1  # кодируем число на 1 больше чтобы могли подавать нули
    i = math.floor(math.log(number, 2))
    return [0]*i + [int(c) for c in "{0:b}".format(number)[-(i+1):]]


def gamma_code_array(bits, DIGITS_IN_NUBER):
    res = []
    for iteration in range(len(bits)//DIGITS_IN_NUBER):
        number_bits = bits[iteration *
                           DIGITS_IN_NUBER:(iteration+1)*DIGITS_IN_NUBER]
        res.extend(gamma_code(number_bits))
    return res


Best DIGITS_IN_NUBER

In [13]:
sorted([(len(gamma_code_array(exceptions_bitmap, din)), din)
  for din in [2, 4, 5,6,7, 8,9, 10, 16, 32, 64, 128, 256, 512]])

[(157007, 7),
 (158672, 8),
 (160494, 6),
 (162036, 9),
 (166105, 10),
 (168731, 5),
 (186648, 4),
 (207147, 16),
 (313225, 2),
 (336944, 32),
 (544465, 64),
 (779832, 128),
 (956033, 256),
 (1056738, 512)]

In [14]:
# len(bitmap) / 
len(gamma_code_array(bitmap, DIGITS_IN_NUBER=8))/(8*1024)


19.3692626953125

In [15]:
# cntr_by_context = 
# char_cnt_by_context = {k: len(v) for k, v in cntr_by_context.items()}
# char_cnt_by_context


### Необходимый объем памяти для хранения всех моделей

Кодируем 21 бит - контекст длины 3. Далее идет неопределенное количество пар символ-счетчик.
Символ кодируем 7 битами. Счетчик по посхеме с переполнением. Когда пары закончились, нужно просигнализировать о том, что дальше пойдет следующая контекстная модель, а не очередной символ в текущей модели. Запишем 7 нулей.

Кодирование с переполнением по схеме 4+8+8+.... Самое большое значение, которое нужно закодировать - 15658. - Счетчик для "e" в контексте " th"

Всевозможные символы

In [16]:
# sorted(list(set(ord(x) for x in decap)))


In [17]:
# SCH = [8, 16, 16]



In [18]:
# Counter(overflow_code(65790, [8, 16, 16]))


In [19]:
ord('a'),chr(97)
# basic latin - 32...127


(97, 'a')

Буквы английского алфавита, стандартные знаки препинания, переводы строк и табы помещаются в первые 128 символов юникода

Хватит 7 бит для того, чтобы закодировать символ.

In [20]:
len(re.findall(' the', text))


7189

In [21]:
np.quantile(cnt_values, q=0.75)


NameError: name 'cnt_values' is not defined

In [21]:
len(sorted([(ord(x),x) for x in set(text)]))


77

In [22]:
# SCH = [8, 16, 16]

def get_context_models(decap) -> defaultdict:
    cntr_by_context = {}
    for c_beg in range(len(decap)-3):
        context = decap[c_beg:c_beg+3]
        folowing = decap[c_beg+3]
        if context not in cntr_by_context:
            cntr_by_context[context] = defaultdict(int)
        cntr_by_context[context][folowing] += 1
    return cntr_by_context


def overflow_code(x, scheme):
    """scheme == [4, 8, 8] ~ 4 + 8 + 8"""
    exp = scheme[0]
    upper_bound = 2**exp - 2
    if 0 <= x <= upper_bound:
        return "{0:b}".format(x).rjust(exp, '0')
    ones_cnt = exp  # кол-во единиц перед началом числа
    for exp in scheme[1:]:
        prev_upper_bound = upper_bound
        upper_bound = upper_bound + 2**exp - 1
        if x <= upper_bound:
            return '1'*ones_cnt + "{0:b}".format(x-(prev_upper_bound+1)).rjust(exp, '0')
        ones_cnt += exp
    raise ValueError(f"Scheme {scheme} is not enough to code {x}")


def overflow_decode(s, scheme):
    cum_sum_ones_read = 0  # сколько суммарно читается единиц
    lower_bound = 0  # сколько надо вычитать
    for exp_ind in range(len(scheme)):
        potentially_read = cum_sum_ones_read + scheme[exp_ind]
        if s.startswith('1'*potentially_read):
            cum_sum_ones_read = potentially_read
            lower_bound += 2**scheme[exp_ind] - 1
        else:
            return (int(s[cum_sum_ones_read:cum_sum_ones_read+scheme[exp_ind]], 2)
                    + lower_bound)


def test_overglow(SCH = [4, 8, 16]):
    for i in range(16000):
        encoded = overflow_code(i, SCH)
        decoded = overflow_decode(encoded, SCH)
        assert i == decoded


def code_string(s):
    return ''.join(["{0:b}".format(ord(c)).rjust(7, '0') for c in s])


def encode_context_models(cntr_by_context, sch):
    result = ''
    for context, cntr in cntr_by_context.items():
        model_encoded = ''
        model_encoded += code_string(context)
        for c, cnt in cntr.items():
            model_encoded += code_string(c)
            model_encoded += overflow_code(cnt, sch)
            # model_encoded += '0'*7
        result += model_encoded
    return result

# SCH = [8, 16,16]
# cntr_by_context = get_context_models(d)



In [23]:
def get_all_counters(cntr_by_context):
    cnt_values = []
    for context, cntr in cntr_by_context.items():
        for c in cntr:
            cnt_values.append(cntr[c])
            if cntr[c] == 15658:
                print(f"CONTEXT={context} {c}")
    return cnt_values

cntr_by_context = get_context_models(decap)
cnt_values = get_all_counters(cntr_by_context)
# fig = px.histogram(cnt_values)
# fig.show()
# как часто встречаются значения-счетчики символов. 75% значений счетчиков не превосходит 26
# значит будет выгодно кодировать сетчики по схеме с переполнением


In [24]:
models = get_context_models(decap)
print(f"{sys.getsizeof(models)/(1024**2)} mB")

0.28133392333984375 mB


In [25]:
len(models[' ta'])

12

In [31]:
bytes_for_lists_of_met_symbols = sum([len(set(cntr.keys())) for cntxt, cntr in models.items()])
bytes_for_lists_of_met_symbols




28238

In [33]:
 sum([len(set(cntr.keys()))*4 for cntxt, cntr in models.items()])

112952

In [35]:
(21294 + 112952 +28392 + 28238 + 112952)/1024

296.70703125

In [30]:
bytes_for_m = len(models.keys()) * 4
bytes_for_m

28392

In [27]:
bytes_for_contexts = len(models.keys())*3 
bytes_for_cntrs = len(cnt_values) * 4 # int - 4 bytes
bytes_for_symbols = len(cnt_values)
f"{(bytes_for_contexts + bytes_for_cntrs + bytes_for_symbols)/(1024**2)} mB"

'0.15495681762695312 mB'

In [28]:
bytes_for_cntrs

112952

In [36]:
sys.getsizeof(models)/1024

288.0859375

In [94]:
# pd.Series(cnt_values).describe()


In [102]:
test_overglow([4,8,16])

In [104]:
sc = [4,8,16]

decap, _ = TextPreprocessor().decap(text)
cntr_by_context=get_context_models(decap)
encoded_cm=encode_context_models(cntr_by_context, sc)
print(f'{sc} {len(encoded_cm)/(8*1024)}')

[4, 8, 16] 63.20751953125


### Choose the best scheme

In [90]:
for sc in [
    [2,8,16],
    [4,8,16],
    [4,4,16],
    [4,16,16],
    [4,6,16],
    [8,8,16],
]:
    decap, _ = TextPreprocessor().decap(text)
    cntr_by_context=get_context_models(decap)
    encoded_cm=encode_context_models(cntr_by_context, sc)
    print(f'{sc} {len(encoded_cm)/(8*1024)}')

[2, 8, 16] 65.7939453125
[4, 8, 16] 63.20751953125
[4, 4, 16] 67.0263671875
[4, 16, 16] 69.25146484375
[4, 6, 16] 63.883544921875
[8, 8, 16] 70.37060546875


In [43]:

# best=None
# best_vol = 10*18
# for sc in [
#     [x,y,16] for x in range(2,17) for y in range(x+1, 17)
# ]:
#     decap, _ = TextPreprocessor().decap(text)
#     cntr_by_context=get_context_models(decap)
#     encoded_cm=encode_context_models(cntr_by_context, sc)
#     if len(encoded_cm)< best_vol:
#         best_vol=len(encoded_cm)
#         best = sc
#     print(f'{sc} {len(encoded_cm)/(8*1024)}')

# print(f"BEST {best}")